## Import libraries

In [1]:
from geo_utils import *

### Load settlement data, select the hamlet part

In [2]:
# Specify country code so all output file will be labeled correctly
country_alpha_3_code = 'ZMB'

# Specify UTM zone EPSG code for this country, so that projection operation in this notebook knows the target CRS
country_utm_epsg = 'epsg:32735'

In [28]:
settlement_extents = gpd.read_feather('./data/ZMB_grid3_settlement_extents_20201222.feather')

settlement_extents.to_crs(country_utm_epsg, inplace=True)

hamlet_settlements = settlement_extents.query('type == "hamlet"')[['mgrs_code','geometry']]

In [29]:
hamlet_settlements = explode_geometry(hamlet_settlements)

hamlet_settlements = add_buffer_column(hamlet_settlements, -40, proj2=None, replace=True)

hamlet_settlements = explode_geometry(hamlet_settlements)

### Add BP column

In [14]:
building_presence_layer = read_csv_as_gpd('./data/ZM_geos_L1_2019.csv', 
                                          id_column = 'sid', 
                                          lon_lat_columns = ['lon','lat'], 
                                          attribute_columns = ['bp'], 
                                          drop_rows_where_duplicates_in_columns = ['observer','lat','lon'], 
                                          drop_rows_where_nan_in_columns = ['lat','lon','bp'])

In [16]:
building_presence_layer.to_crs(country_utm_epsg, inplace=True)

building_presence_layer = add_buffer_column(building_presence_layer, buffer_radius = 100, cap_style = 'square', replace=True)

hamlet_settlements = left_spatial_join(hamlet_settlements, building_presence_layer[['bp','geometry']])

hamlet_settlements['bp_value'] = hamlet_settlements['bp'].map({'Y':1,'N':0})

### Add HRSL column

In [17]:
hamlet_settlements, centroid_column = add_centroid_column(hamlet_settlements, return_new_column=True)

hamlet_settlements[centroid_column] = hamlet_settlements[centroid_column].to_crs('epsg:4326')


In [20]:

hamlet_settlements = get_raster_point_value(hamlet_settlements, centroid_column, raster_filepath = './data/ZMB_HRSL_hasim_20211222.tif', new_column = 'hrsl_value')

# hamlet_settlements = hamlet_settlements.drop([centroid_column], axis=1)

### Add WSF column

In [30]:
hamlet_settlements.to_crs('epsg:4326', inplace=True)

In [31]:
wsf_2019_geotiff_filepath_list = glob('./data/WSF2019/*.tif')

In [32]:
hamlet_settlements = add_covering_geotiff_column(hamlet_settlements, 
                                                 geom_column = 'geometry',
                                                 geotiff_filepath_column = 'wsf2019_geotiff_filepath',
                                                 geotiff_filepath_list = wsf_2019_geotiff_filepath_list)

In [36]:
hamlet_settlements = get_raster_value_distribution(hamlet_settlements, 
                                                   geom_column = 'geometry',
                                                   uuid_column = 'mgrs_code',
                                                   geotiff_filepath_column = 'wsf2019_geotiff_filepath',
                                                   geotiff_filepath_list = wsf_2019_geotiff_filepath_list, 
                                                   code_to_label_mapping = {0:'no_settlement',255:'settlement'},
                                                   label_marker = 'wsf2019')

100%|███████████████████████████████████████████| 56/56 [01:06<00:00,  1.18s/it]


In [38]:
hamlet_settlements = drop_bounds_columns(hamlet_settlements)

In [41]:
hamlet_settlements

,mgrs_code,geometry,wsf2019_geotiff_filepath,wsf2019__no_settlement,wsf2019__settlement
0,35LLG6741_02,"POLYGON ((25.78240 -12.28583, 25.78258 -12.285...",./data/WSF2019/WSF2019_v1_24_-14.tif,3239,119
1,36LUM0662_05,"POLYGON ((31.21574 -12.09164, 31.21604 -12.091...",./data/WSF2019/WSF2019_v1_30_-14.tif,3488,100
2,36LUM0662_05,"POLYGON ((31.21574 -12.09164, 31.21604 -12.091...",./data/WSF2019/WSF2019_v1_30_-14.tif,34,2
3,36LUM0662_05,"POLYGON ((31.21574 -12.09164, 31.21604 -12.091...",./data/WSF2019/WSF2019_v1_30_-14.tif,34,1
4,36LSM7733_01,"POLYGON ((30.03934 -12.34972, 30.03876 -12.349...",./data/WSF2019/WSF2019_v1_30_-14.tif,3900,180
...,...,...,...,...,...
726599,36LUK1417_03,"POLYGON ((31.28305 -14.30617, 31.28306 -14.306...",./data/WSF2019/WSF2019_v1_30_-16.tif,903,69
726600,36LUK1417_03,"POLYGON ((31.28305 -14.30617, 31.28306 -14.306...",./data/WSF2019/WSF2019_v1_30_-16.tif,107,3
726601,36LUK1417_03,"POLYGON ((31.28305 -14.30617, 31.28306 -14.306...",./data/WSF2019/WSF2019_v1_30_-16.tif,0,0
726602,35LPD6904_05,"POLYGON ((28.57539 -15.32678, 28.57539 -15.326...",./data/WSF2019/WSF2019_v1_28_-16.tif,1110,45


### Add Google column

In [12]:
google_buildings_layer = read_csv_as_gpd('./data/ZMB_google_buildings_footprint.csv', 
                                          id_column = None, 
                                          lon_lat_columns = ['longitude','latitude'], 
                                          drop_rows_where_duplicates_in_columns = ['longitude','latitude']
                                        )

In [13]:
google_buildings_layer = left_spatial_join(google_buildings_layer, hamlet_settlements[['mgrs_code','geometry']])

In [14]:
settlements_with_google_builing = google_buildings_layer.dropna()['mgrs_code'].unique().tolist()

hamlet_settlements.loc[hamlet_settlements['mgrs_code'].isin(settlements_with_google_builing),'google_value'] = 1

hamlet_settlements['google_value'] = hamlet_settlements['google_value'].fillna(0)